In [ ]:
from google.colab import files
import csv
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np

!pip install py-ecg-detectors
from ecgdetectors import Detectors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 554 kB 49.3 MB/s 
  Created wheel for gatspy: filename=gatspy-0.3-py3-none-any.whl size=43818 sha256=d06d421377f80db04a3af8a981622828346e3b732d5da0449caeecc8360227cb
  Stored in directory: /root/.cache/pip/wheels/1f/c1/4c/8de278f816259b4f67e2356c0bda260a593375661e400bf31f
Successfully built gatspy


In [ ]:
#plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]

#plt.rcParams["figure.figsize"] = (50,20)

### Data Preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/drive/My Drive/DD/Matérias/3/Med/training.zip" .
!unzip training.zip

A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: training/train_ecg_01001.mat  
  inflating: training/train_ecg_01002.mat  
  inflating: training/train_ecg_01003.mat  
  inflating: training/train_ecg_01004.mat  
  inflating: training/train_ecg_01005.mat  
  inflating: training/train_ecg_01006.mat  
  inflating: training/train_ecg_01007.mat  
  inflating: training/train_ecg_01008.mat  
  inflating: training/train_ecg_01009.mat  
  inflating: training/train_ecg_01010.mat  
  inflating: training/train_ecg_01011.mat  
  inflating: training/train_ecg_01012.mat  
  inflating: training/train_ecg_01013.mat  
  inflating: training/train_ecg_01014.mat  
  inflating: training/train_ecg_01015.mat  
  inflating: training/train_ecg_01016.mat  
  inflating: training/train_ecg_01017.mat  
  inflating: training/train_ecg_01018.mat  
  inflating: training/train_ecg_01019.mat  
  inflating: training/train_ecg_01020.mat  
  inflating: training/train_ecg_01021.mat  
  inflating: trai

In [ ]:
#TODO make array creation smarter
#TODO make tilde array
fs = 300                                                  # Sampling-Frequenz 300 Hz
detectors = Detectors(fs)                                 # Initialisierung des QRS-Detektors
arr = np.zeros((6000,4))
diagnosis = np.zeros(6000,dtype='str')
with open('training/REFERENCE.csv') as csv_file:      # Einlesen der Liste mit Dateinamen und Zuordnung
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
      data = sio.loadmat('training/'+row[0]+'.mat')   # Import der EKG-Dateien
      ecg_lead = data['val'][0]
      arr[line_count][0] = line_count
      r_peaks = detectors.hamilton_detector(ecg_lead)
      sdnn = np.std(np.diff(r_peaks)/fs*1000)
      arr[line_count][1] = sdnn
      r_peaks = detectors.two_average_detector(ecg_lead)
      sdnn = np.std(np.diff(r_peaks)/fs*1000)
      arr[line_count][2] = sdnn
      r_peaks = detectors.swt_detector(ecg_lead)
      sdnn = np.std(np.diff(r_peaks)/fs*1000)
      arr[line_count][3] = sdnn
      if row[1]=='N':
        diagnosis[line_count] = 'N'         # Zuordnung zu "Normal"
      if row[1]=='A':
        diagnosis[line_count] = 'A'
      if row[1]=='O':
        diagnosis[line_count] = 'O'
      if row[1]=='~':
        diagnosis[line_count] = '~'
      line_count = line_count + 1
      if (line_count % 1000)==0:
        print(str(line_count) + "\t Dateien wurden verarbeitet.")

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in true_divide
  subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1000	 Dateien wurden verarbeitet.
2000	 Dateien wurden verarbeitet.
3000	 Dateien wurden verarbeitet.
4000	 Dateien wurden verarbeitet.
5000	 Dateien wurden verarbeitet.
6000	 Dateien wurden verarbeitet.


## Importing into Pandas

In [ ]:
import pandas as pd

df = pd.DataFrame(arr, columns = ['index', 'ham', 'two', 'swt'])
df.drop(['index'],axis=1, inplace = True)
df.rename(columns = {'level_0':'index'}, inplace = True)

df['diagnosis'] = diagnosis
df1=df.copy()
df2=df.copy()

In [ ]:
df1.loc[df['ham']==0]=df1['ham'].mean()
df1['ham'].fillna(df1['ham'].mean(),inplace=True)
df1['swt'].fillna(df1['swt'].mean(),inplace=True)

In [ ]:
df2=df.loc[(df['ham']!=0)].copy()
df2.dropna(inplace=True)

In [ ]:
df1.to_csv('meanCorrected.csv', index=False)
df2.to_csv('dropNaN.csv', index=False)
df2

,ham,two,swt,diagnosis
0,150.386747,333.387092,58.673333,N
1,52.476203,52.503517,58.838725,N
2,93.402127,90.766423,99.645296,O
3,133.532649,28.331410,48.052787,N
4,108.211430,113.671689,103.825883,O
...,...,...,...,...
5995,226.645859,177.905119,165.673165,~
5996,65.954848,104.869593,50.639727,A
5997,42.410626,371.868917,159.673449,O
5998,301.048194,252.627208,330.682805,O
